# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create a list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/troy/Documents/data-engineering/Projects/data-modeling-cassandra


#### Process the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
def rows(file):
    """Process the rows of the specified csv file.
    
    Args:
        file (string): csv file to process.
    
    Yields:
        list: The next row of the file.
    """
    # Read the csv file 
    with open(file, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Create a csv reader object
        csvreader = csv.reader(csvfile) 
        next(csvreader)  # Skip the header

        for line in csvreader:
            yield line
            
# Create a smaller event data csv file with fewer columns called event_datafile_full.csv
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for file in file_path_list:
        for row in rows(file):
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], \
                             row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing the Apache Cassandra code in the cells below

#### Create a Cluster

In [5]:
# Make a connection to a Cassandra instance on the local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create KEYSPACE if it doesn't exist
try:
    session.execute("""CREATE  KEYSPACE IF NOT EXISTS udacity 
                       WITH REPLICATION =
                       { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Create tables to run the three queries.

### Query 1:  Return the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Use the sessionId as the partition key and the itemInSession as the clustering column

In [8]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS sessions (
                       session_id bigint, item_in_session int, artist text, song_title text, length decimal,
                       PRIMARY KEY (session_id, item_in_session))
                    """)
except Exception as e:
    print(e)        

In [9]:
from decimal import Decimal

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skip header
    for line in csvreader:
        query = "INSERT INTO sessions (session_id, item_in_session, artist, song_title, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
try:
    rows = session.execute("""SELECT artist, song_title, length
                              FROM sessions
                              WHERE session_id = 338 AND item_in_session = 4""")
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows), columns=['artist', 'song_title', 'length'])
print(df.to_string())

      artist                       song_title    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073


### Query 2: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Use a composite partition key consisting of the userId and the sessionId and the itemInSession as the clustering column to sort the results

#### Combine the first and last name into a single column to simplify the table

In [11]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS user_sessions (
                       user_id bigint, session_id bigint, item_in_session int,
                       artist text, song text, user text,
                       PRIMARY KEY ((user_id, session_id), item_in_session))
                    """)
except Exception as e:
    print(e)        

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (user_id, session_id, item_in_session, artist, song, user)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], \
                                line[9], ' '.join([line[1], line[4]])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
try:
    rows = session.execute("""SELECT artist, song, user
                              FROM user_sessions
                              WHERE user_id = 10 AND session_id = 182""")
except Exception as e:
    print(e)
 
df = pd.DataFrame(list(rows), columns=['Artist', 'Song', 'User'])
print(df.to_string())

              Artist                                               Song         User
0   Down To The Bone                                 Keep On Keepin' On  Sylvie Cruz
1       Three Drives                                        Greece 2000  Sylvie Cruz
2  Sebastien Tellier                                          Kilometer  Sylvie Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...  Sylvie Cruz


### Query 3: Return every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

#### Use the song title as the partition key and the userId as the clustering column

#### Combine the first and last name into a single column to simplify the table

In [14]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS song_plays (
                       song text, user_id bigint, user text,
                       PRIMARY KEY (song, user_id))
                    """)
except Exception as e:
    print(e) 

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_plays (song, user_id, user)"
        query = query + " VALUES(%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), ' '.join([line[1], line[4]])))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
try:
    rows = session.execute("""SELECT user
                              FROM song_plays
                              WHERE song = 'All Hands Against His Own'""")
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows), columns=['User'])
print(df.to_string())

               User
0  Jacqueline Lynch
1      Tegan Levine
2      Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE IF EXISTS sessions")
session.execute("DROP TABLE IF EXISTS user_sessions")
session.execute("DROP TABLE IF EXISTS song_plays")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()